In [1]:
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

#from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.datasets import mnist
from spektral.layers import GatedGraphConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv # as GraphConv
from spektral.layers import GCNConv # as GraphConv

from spektral.utils.convolution import gcn_filter # For GCNConv
from spektral.utils.convolution import normalized_adjacency # For GCSConv

import numpy as np
import pandas as pd

In [2]:
# Parameters
l2_reg = 5e-4         # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32       # Batch size
epochs = 5        # Number of training epochs
es_patience = 200     # Patience fot early stopping

In [3]:
# load the proximiity_matrix.csv
adj = np.loadtxt('adjacency_matrix.csv', delimiter=',')
print(adj.shape)




(468, 468)


In [4]:
# carrega os nós
nodes = np.loadtxt('vertices.csv', delimiter=',')
print(nodes.shape)


(468,)


In [5]:
# carrega os edges
edges = np.loadtxt('edges.csv', delimiter=',')
print(edges.shape)


(2556, 2)


In [6]:
# carrega as distancias
y = np.loadtxt('distances.csv', delimiter=',')
print(y.shape)

(2556,)


In [7]:
# cria um dataframe com os dados
df = pd.DataFrame({'source': edges[:, 0], 'target': edges[:, 1], 'weight': y})
df.head()

,source,target,weight
0,18.0,17.0,1.650424
1,82.0,38.0,0.962110
2,8.0,9.0,6.746557
3,456.0,248.0,3.331333
4,167.0,2.0,6.610048


In [8]:
# dimension of the target
n_out = 6

N = len(nodes)  # Number of nodes in the graph
F = 1  # Dimension of node features


In [9]:
fltr = normalized_laplacian(adj) # For GCNConv
# fltr = gcn_filter(adj) # For GCNConv
# fltr = normalized_adjacency(adj) # For GCSConv

https://graphneural.network/layers/convolution/#gcsconv

In [10]:
# Prepare data
fltr = normalized_adjacency(adj)

# Model using GCSConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCSConv(32,
                          activation='elu',
                          kernel_regularizer=l2(l2_reg),
                          use_bias=True)([X_in, A_in])
graph_conv_2 = GCSConv(32,
                            activation='elu',
                            kernel_regularizer=l2(l2_reg),
                            use_bias=True)([graph_conv_1, A_in])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation='relu')(flatten)
output = Dense(n_out, activation='softmax')(fc)

# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()


c:\Users\rodri\anaconda3\envs\mirror\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 468, 1)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(468, 468)]                 0         []                            
                                                                                                  
 gcs_conv (GCSConv)          (None, 468, 32)              96        ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 gcs_conv_1 (GCSConv)        (None, 468, 32)              2080      ['gcs_conv[0][0]',        

https://graphneural.network/layers/convolution/#gcnconv

In [11]:
# Prepare data
fltr = gcn_filter(adj)
# Model using GCNConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCNConv(32,
                            activation='elu',
                            kernel_regularizer=l2(l2_reg),
                            use_bias=True)([X_in, A_in])
graph_conv_2 = GCNConv(32,
                            activation='elu',
                            kernel_regularizer=l2(l2_reg),
                            use_bias=True)([graph_conv_1, A_in])
# removed the flatten layer because it is not necessary for GCNConv
fc = Dense(512, activation='relu')(graph_conv_2)
output = Dense(n_out, activation='softmax')(fc)


# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 468, 1)]             0         []                            
                                                                                                  
 input_4 (InputLayer)        [(468, 468)]                 0         []                            
                                                                                                  
 gcn_conv (GCNConv)          (None, None, 32)             64        ['input_3[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 gcn_conv_1 (GCNConv)        (None, None, 32)             1056      ['gcn_conv[0][0]',      